In [ ]:
#@title ### Mount Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@title ### Prepare Dependencies
%%capture
!git clone --branch updt https://github.com/TheLastBen/diffusers
!pip install -q diffusers
!pip install -U accelerate
!pip install -U transformers
!pip install -U wget
!pip install -U bitsandbytes
!pip install -U xformers
!pip install -U triton

In [ ]:
%%capture
from pathlib import Path
import wget
#@title ### Model Selection

MODEL_NAME = "nitrosocke/archer-diffusion" #@param ["runwayml/stable-diffusion-v1-5", "nitrosocke/Arcane-Diffusion", "nitrosocke/mo-di-diffusion", "nitrosocke/spider-verse-diffusion", "nitrosocke/Ghibli-Diffusion", "nitrosocke/archer-diffusion", "nitrosocke/classic-anim-diffusion", "DGSpitzer/Cyberpunk-Anime-Diffusion", "dallinmackay/Tron-Legacy-diffusion", "Envvi/Inkpunk-Diffusion", "dallinmackay/Van-Gogh-diffusion", "ogkalu/Comic-Diffusion"]
CLASS_NAME = 'person' #@param ["person", "man", "woman"]
INSTANCE_TOKEN = 'abotrika' #@param{type: 'string'}

#===============================================================================

class_path = Path('./regularization_images')
if not class_path.exists():
    class_path.mkdir(parents=True)

output_path = Path('./output').joinpath(INSTANCE_TOKEN).joinpath(MODEL_NAME)
if not output_path.exists():
    output_path.mkdir(parents=True)

CLASS_DIR = class_path.as_posix()
OUTPUT_DIR = output_path.as_posix()

#===============================================================================

if CLASS_NAME == 'person':
    data_file = 'person_v1-5_mse_vae_ddim50_cfg7_n2115'
elif CLASS_NAME == 'man':
    data_file = 'guy_v1-5_mse_vae_ddim50_cfg7_n4820'
elif CLASS_NAME == 'woman':
    data_file = 'woman_v1-5_mse_vae_ddim50_cfg7_n4420'

url = f'https://huggingface.co/datasets/ProGamerGov/StableDiffusion-v1-5-Regularization-Images/resolve/main/{data_file}.zip'

file = wget.download(url=url, out=f'{CLASS_DIR}')

filename = Path(file).name.replace('.zip', '')

!unzip -q -o -d $CLASS_DIR $file 
!rm $file

#===============================================================================

class_path = class_path.joinpath(filename)
CLASS_DIR = class_path.as_posix()

In [ ]:
from google.colab import files
import shutil
import gdown
import os

#@title ### Add Google Drive URL that contains the images you want to use.
#@markdown - Leave the URL empty to upload the files directly.

instances_path = Path('./instance_images')
if not instances_path.exists():
    instances_path.mkdir(parents=True)

INSTANCE_DIR = instances_path.as_posix()

GDRIVE__URL = "https://drive.google.com/drive/folders/1czBGt198hM_cGtareNj2SF30KHV8PhTK?usp=share_link" #@param {type:"string"}


if len(GDRIVE__URL) > 0:    
    gdown.download_folder(GDRIVE__URL, quiet=True, use_cookies=False, output=INSTANCE_DIR)
    for i, img in enumerate(os.listdir(INSTANCE_DIR)):
        os.rename(instances_path.joinpath(img).as_posix(), 
                  instances_path.joinpath(f'{INSTANCE_TOKEN}-({i}).png').as_posix())
else:
    imgs = files.upload()
    for i, img in enumerate(imgs.keys()):
        p = instances_path.joinpath(f'{INSTANCE_TOKEN}-({i}).png')
        shutil.move(img, p.as_posix())


In [ ]:
import random

#@title ### Training

TRAINING_STEPS = 3000 #@param {type:"integer"}
CLASS_IMAGES_NUM = 200 #@param {type:"integer"}
SEED = None #@param {type:"number"}
if SEED is None:
    SEED = random.randint(1, 999999)

#===============================================================================

steps = TRAINING_STEPS // 10
!accelerate launch --mixed_precision="fp16" diffusers/examples/dreambooth/train_dreambooth.py \
    --image_captions_filename \
    --train_text_encoder \
    --dump_only_text_encoder \
    --pretrained_model_name_or_path="$MODEL_NAME" \
    --instance_data_dir="$INSTANCE_DIR" \
    --class_data_dir="$CLASS_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --with_prior_preservation --prior_loss_weight=1.0 \
    --mixed_precision="fp16" \
    --seed=$SEED \
    --resolution=512 \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 --gradient_checkpointing \
    --use_8bit_adam \
    --learning_rate=2e-6 \
    --lr_scheduler="polynomial" \
    --lr_warmup_steps=0 \
    --max_train_steps=$steps \
    --num_class_images=200

#===============================================================================

!accelerate launch --mixed_precision="fp16" diffusers/examples/dreambooth/train_dreambooth.py \
    --image_captions_filename \
    --train_only_unet \
    --save_starting_step=5000 \
    --save_n_steps=5000 \
    --pretrained_model_name_or_path="$MODEL_NAME" \
    --instance_data_dir="$INSTANCE_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --seed=$SEED \
    --resolution=512 \
    --mixed_precision="fp16" \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 --gradient_checkpointing \
    --use_8bit_adam \
    --learning_rate=2e-6 \
    --lr_scheduler="polynomial" \
    --lr_warmup_steps=0 \
    --max_train_steps=$TRAINING_STEPS

In [ ]:
#@title Save Checkpoint to Google Drive
SAVE_LOCATION = "My Stable Diffusion/Models" #@param {type:"string"}

ckpt_path = Path(f'/content/gdrive/MyDrive/{SAVE_LOCATION}')
if not ckpt_path.exists():
    ckpt_path.mkdir(parents=True)

ckpt_path = ckpt_path.joinpath(f'{INSTANCE_TOKEN}_archer.ckpt')
CKPT_DIR = ckpt_path.as_posix()

!python diffusers/scripts/convert_diffusers_to_original_stable_diffusion.py --model_path $OUTPUT_DIR  --checkpoint_path "$CKPT_DIR" --half